In [2]:
import pandas as pd
from cafu.queries.partida import Partida
from cafu.metadata import path
path_datalake = path('datalake')

In [3]:
path_camp = path_datalake+'/jogos_ids/franca/2019-2020.csv'
df = pd.read_csv(path_camp)

In [4]:
df

,jogo_id,dates,rodada,time_casa,time_visitante
0,541855,2020-05-23,38,Stade Rennais,AS Mônaco
1,541856,2020-05-23,38,Metz,Angers
2,541857,2020-05-23,38,Lyon,Brest
3,541858,2020-05-23,38,Saint-Etienne,Dijon FCO
4,541859,2020-05-23,38,Bordeaux,Lille
...,...,...,...,...,...
375,542230,2019-08-10,1,Nice,SC Amiens
376,542231,2019-08-10,1,Dijon FCO,Saint-Etienne
377,542232,2019-08-10,1,Montpellier,Stade Rennais
378,542233,2019-08-10,1,Marseille,Stade de Reims


In [7]:
import findspark
findspark.init('/usr/local/spark/spark-3.0.3-bin-hadoop2.7') 

In [4]:
#import findspark
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

# Find Spark Locally
#location = findspark.find()
#findspark.init(location, edit_rc=True)

from cafu.utils import get_spark

In [5]:
from cafu.utils import get_spark

In [6]:
spark = get_spark(1)

22/03/14 18:56:08 WARN Utils: Your hostname, luan-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/03/14 18:56:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/luan/.ivy2/cache
The jars for the packages stored in: /home/luan/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/spark/spark-3.0.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7b0cf624-c003-442e-8d5e-e52cef7da7b5;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in cen

In [7]:
spark

In [9]:
spark.createDataFrame(df).limit(1).show()

+-------+----------+------+-------------+--------------+
|jogo_id|     dates|rodada|    time_casa|time_visitante|
+-------+----------+------+-------------+--------------+
| 541855|2020-05-23|    38|Stade Rennais|     AS Mônaco|
+-------+----------+------+-------------+--------------+



In [7]:
id_ = 57299

In [8]:
req = Partida(id_, check_status=False)
campeonato = req.campeonato()
times = req.nomes_times()
date = req.day()

In [9]:
date

In [10]:
times

In [11]:
campeonato